In [1]:
import random
import numpy as np

In [2]:
import random
import numpy as np


def calculate_z(n, x):
    return sum(n_i * x_i for n_i, x_i in zip(n, x))


def is_valid(n, M, a, b, c, d):
    n1, n2, n3, n4 = n
    n4_lower_bound = max(0, d - (n1 - a + n2 - b + n3 - c))
    n4_upper_bound = M - (n1 - a + n2 - b + n3 - c)
    return (a <= n1 <= M and
            (b - (n1 - a)) <= n2 <= M - (n1 - a) and
            (c - (n1 - a + n2 - b)) <= n3 <= M - (n1 - a + n2 - b) and
            n4_lower_bound <= n4 <= n4_upper_bound and
            sum(n) == a + b + c + d)


## unused

Fungsi calculate_z(n, x) digunakan untuk menghitung nilai objektif (fitness) dari suatu solusi atau partikel. Nilai ini merupakan hasil perkalian antara jumlah unit dari masing-masing item (n1, n2, n3, n4) dengan harga satuannya (x1, x2, x3, x4), kemudian dijumlahkan untuk mendapatkan total biaya atau nilai Z yang ingin diminimalkan atau dimaksimalkan.

In [3]:

def generate_random_solution(M, a, b, c, d):
    while True:
        n1 = random.randint(a, M)
        n2 = random.randint(max(b - (n1 - a), 0), M - (n1 - a))
        n3 = random.randint(max(c - (n1 - a + n2 - b), 0), M - (n1 - a + n2 - b))
        n4 = (a + b + c + d) - n1 - n2 - n3
        if is_valid([n1, n2, n3, n4], M, a, b, c, d):
            return [n1, n2, n3, n4]


def particle_swarm_optimization(x, M, a, b, c, d, num_particles=50, num_iterations=100, w=0.5, c1=1.5, c2=1.5):
    
    particles = [generate_random_solution(M, a, b, c, d) for _ in range(num_particles)]
    velocities = [np.random.uniform(-1, 1, 4) for _ in range(num_particles)]
    personal_best_positions = particles.copy()
    personal_best_scores = [calculate_z(p, x) for p in particles]
    global_best_position = min(particles, key=lambda p: calculate_z(p, x))
    global_best_score = calculate_z(global_best_position, x)

    for _ in range(num_iterations):
        for i, particle in enumerate(particles):
            velocities[i] = (w * velocities[i] +
                             c1 * random.random() * (np.array(personal_best_positions[i]) - np.array(particle)) +
                             c2 * random.random() * (np.array(global_best_position) - np.array(particle)))
            new_position = np.array(particle) + velocities[i]
            new_position = [int(np.clip(new_position[j], a if j == 0 else max(b - (M + particle[0] - a), 0) if j == 1 else max(c - (M + particle[1] - b), 0) if j == 2 else 0, M)) for j in range(4)]
            new_position[3] = (a + b + c + d) - sum(new_position[:3])

            if is_valid(new_position, M, a, b, c, d):
                particles[i] = new_position
                current_score = calculate_z(new_position, x)
                if current_score < personal_best_scores[i]:
                    personal_best_positions[i] = new_position
                    personal_best_scores[i] = current_score
                if current_score < global_best_score:
                    global_best_position = new_position
                    global_best_score = current_score

    return global_best_position, global_best_score


In [5]:

x = [40, 30, 31, 40]
M = 29000
a = 18524
b = 7937
c = 8043
d = 15921

best_solution, best_z = particle_swarm_optimization(x, M, a, b, c, d)

if best_solution:
    best_n1, best_n2, best_n3, best_n4 = best_solution
    print(f"PSO - Nilai n1 terbaik: {best_n1}")
    print(f"PSO - Nilai n2 terbaik: {best_n2}")
    print(f"PSO - Nilai n3 terbaik: {best_n3}")
    print(f"PSO - Nilai n4 terbaik: {best_n4}")
    print(f"PSO - Nilai z terkecil: {best_z}")
else:
    print("Tidak ditemukan solusi yang valid.")


PSO - Nilai n1 terbaik: 18524
PSO - Nilai n2 terbaik: 29000
PSO - Nilai n3 terbaik: 2901
PSO - Nilai n4 terbaik: 0
PSO - Nilai z terkecil: 1700891


In [6]:
def optimasi_harga_perbekalan(a, x, C):
    n = len(a)  # Jumlah tempat yang disinggahi
    M = sum(x)  # Total kebutuhan bekal
    total_harga = 0  # Variabel untuk menyimpan total harga perbekalan
    sisa_bekal = 0  # Variabel untuk menyimpan bekal yang tersisa
    bekal_di_beli = [0] * n  # List untuk menyimpan jumlah bekal yang dibeli di tiap tempat

    for i in range(n):
        # Hitung berapa banyak bekal yang dapat dibeli dengan memperhatikan kapasitas maksimal
        bekal_yang_dibutuhkan = x[i]
        bekal_yang_dapat_dibeli = min(C - sisa_bekal, bekal_yang_dibutuhkan)
        
        # Tambahkan jumlah bekal yang dibeli ke total sisa bekal
        sisa_bekal += bekal_yang_dapat_dibeli
        
        # Simpan jumlah bekal yang dibeli di tempat ini
        bekal_di_beli[i] = bekal_yang_dapat_dibeli

        # Tambahkan biaya bekal yang dibeli ke total harga
        total_harga += bekal_di_beli[i] * a[i]

        # Kurangi sisa bekal dengan bekal yang dibutuhkan
        sisa_bekal -= bekal_yang_dibutuhkan

        # Pastikan sisa bekal tidak melebihi kapasitas
        sisa_bekal = min(sisa_bekal, C)

    return total_harga, bekal_di_beli

# Contoh penggunaan
a = [5, 2, 4, 3]  # Daftar harga perbekalan di tiap tempat
x = [3, 1, 4, 2]  # Kebutuhan bekal dari satu tempat ke tempat berikutnya
C = 10  # Kapasitas maksimal bekal yang dapat dibawa

total_harga, bekal_di_beli = optimasi_harga_perbekalan(a, x, C)

print(f"Total harga perbekalan: {total_harga}")
print("Bekal yang harus dibeli di tiap tempat:")
for i in range(len(a)):
    print(f"Tempat {i+1}: {bekal_di_beli[i]} unit")


Total harga perbekalan: 39
Bekal yang harus dibeli di tiap tempat:
Tempat 1: 3 unit
Tempat 2: 1 unit
Tempat 3: 4 unit
Tempat 4: 2 unit


In [7]:
import random
import numpy as np

# Fungsi untuk menghitung nilai z
def calculate_z(n, x):
    return np.dot(n, x)

# Fungsi untuk memeriksa apakah nilai n memenuhi batasan
def is_valid(n, M, constraints):
    n_sum = sum(n)
    constraints_sum = sum(constraints)
    if n_sum != constraints_sum:
        return False

    n_accum = 0
    for i in range(len(constraints)):
        n_accum += n[i] - constraints[i]
        if not (0 <= n_accum <= M):
            return False
    return True

In [8]:
def generate_random_solution(M, constraints):
    num_vars = len(constraints)
    while True:
        n = [random.randint(constraints[i], M) for i in range(num_vars)]
        n[-1] = sum(constraints) - sum(n[:-1])
        if is_valid(n, M, constraints):
            return n

In [9]:
# Fungsi untuk menjalankan algoritma PSO
def particle_swarm_optimization(x, M, constraints, num_particles=50, num_iterations=100, w=0.5, c1=1.5, c2=1.5):
    num_vars = len(constraints)
    
    # Inisialisasi partikel
    particles = [generate_random_solution(M, constraints) for _ in range(num_particles)]
    velocities = [np.random.uniform(-1, 1, num_vars) for _ in range(num_particles)]
    personal_best_positions = particles.copy()
    personal_best_scores = [calculate_z(p, x) for p in particles]
    global_best_position = min(particles, key=lambda p: calculate_z(p, x))
    global_best_score = calculate_z(global_best_position, x)

    for _ in range(num_iterations):
        for i, particle in enumerate(particles):
            r1, r2 = random.random(), random.random()
            velocities[i] = (w * velocities[i] +
                             c1 * r1 * (np.array(personal_best_positions[i]) - np.array(particle)) +
                             c2 * r2 * (np.array(global_best_position) - np.array(particle)))
            new_position = np.array(particle) + velocities[i]
            new_position = [int(np.clip(new_position[j], constraints[j], M)) for j in range(num_vars)]
            new_position[-1] = sum(constraints) - sum(new_position[:-1])

            if is_valid(new_position, M, constraints):
                particles[i] = new_position
                current_score = calculate_z(new_position, x)
                if current_score < personal_best_scores[i]:
                    personal_best_positions[i] = new_position
                    personal_best_scores[i] = current_score
                if current_score < global_best_score:
                    global_best_position = new_position
                    global_best_score = current_score

    return global_best_position, global_best_score

In [13]:
x = [40, 30, 31, 40]
M = 29000
constraints = [18524, 7937, 8043, 15921]

# Temukan nilai n terbaik
best_solution, best_z = particle_swarm_optimization(x, M, constraints)

if best_solution:
    for idx, val in enumerate(best_solution):
        print(f"PSO - Nilai n{idx + 1} terbaik: {val}")
    print(f"PSO - Nilai z terkecil: {best_z}")
else:
    print("Tidak ditemukan solusi yang valid.")

PSO - Nilai n1 terbaik: 18524
PSO - Nilai n2 terbaik: 29000
PSO - Nilai n3 terbaik: 15980
PSO - Nilai n4 terbaik: -13079
PSO - Nilai z terkecil: 1583180


## primary

In [7]:
import random
import numpy as np

# Fungsi untuk menghitung nilai z
def calculate_z(n, x):
    return np.dot(n, x)

# Fungsi untuk memeriksa apakah nilai n memenuhi batasan
def is_valid(n, M, constraints):
    n_sum = sum(n)
    constraints_sum = sum(constraints)
    if n_sum != constraints_sum:
        return False

    n_accum = 0
    for i in range(len(constraints)):
        n_accum += n[i] - constraints[i]
        if not (0 <= n_accum <= M):
            return False
    return True

# Fungsi untuk menghasilkan solusi acak yang valid
def generate_random_solution(M, constraints):
    num_vars = len(constraints)
    while True:
        n = [random.randint(constraints[i], M) for i in range(num_vars - 1)]
        n_sum = sum(n)
        n_last = sum(constraints) - n_sum
        if n_last >= 0:
            n.append(n_last)
            if is_valid(n, M, constraints):
                return n

# Fungsi untuk menjalankan algoritma PSO
def particle_swarm_optimization(x, M, constraints, num_particles=50, num_iterations=100, w=0.5, c1=1.5, c2=1.5):
    num_vars = len(constraints)
    
    # Inisialisasi partikel
    particles = [generate_random_solution(M, constraints) for _ in range(num_particles)]
    velocities = [np.random.uniform(-1, 1, num_vars) for _ in range(num_particles)]
    personal_best_positions = particles.copy()
    personal_best_scores = [calculate_z(p, x) for p in particles]
    global_best_position = min(particles, key=lambda p: calculate_z(p, x))
    global_best_score = calculate_z(global_best_position, x)

    for _ in range(num_iterations):
        for i, particle in enumerate(particles):
            r1, r2 = random.random(), random.random()
            velocities[i] = (w * velocities[i] +
                             c1 * r1 * (np.array(personal_best_positions[i]) - np.array(particle)) +
                             c2 * r2 * (np.array(global_best_position) - np.array(particle)))
            new_position = np.array(particle) + velocities[i]
            new_position = [int(np.clip(new_position[j], 0, M)) for j in range(num_vars)]
            new_position[-1] = sum(constraints) - sum(new_position[:-1])

            if new_position[-1] >= 0 and is_valid(new_position, M, constraints):
                particles[i] = new_position
                current_score = calculate_z(new_position, x)
                if current_score < personal_best_scores[i]:
                    personal_best_positions[i] = new_position
                    personal_best_scores[i] = current_score
                if current_score < global_best_score:
                    global_best_position = new_position
                    global_best_score = current_score

    return global_best_position, global_best_score

# Nilai yang diberikan (inputan)
x = [35, 24, 20, 35] # Harga fw pada setiap ports
M = 27000 # tank
constraints = [6412, 1140, 3394, 6825] # Kebutuhan (fwneeds)

# Temukan nilai n terbaik
best_solution, best_z = particle_swarm_optimization(x, M, constraints)

if best_solution:
    for idx, val in enumerate(best_solution):
        print(f"PSO - Nilai n{idx + 1} terbaik: {val}")
    print(f"PSO - Nilai z terkecil: {best_z}")
else:
    print("Tidak ditemukan solusi yang valid.")


PSO - Nilai n1 terbaik: 6578
PSO - Nilai n2 terbaik: 974
PSO - Nilai n3 terbaik: 10219
PSO - Nilai n4 terbaik: 0
PSO - Nilai z terkecil: 457986
